In [2]:
import os
!pip install pandas
import pandas as pd
!pip install seaborn
import seaborn as sns
!pip installmatplotlib
import matplotlib.pyplot as plt
import requests
!pip install mygene
import mygene
import json
!pip install numpy
import numpy as np

ERROR: unknown command "installmatplotlib"


In [3]:
## disease table
myPath = 'platform/23.02/output/etl/json/diseases'

col_names = ['id', 'name', 'therapeuticAreas']

allFiles = os.listdir(myPath)
first = True
for module in allFiles:
    # get one file in directory
    if 'part-' in module: #if the word module is in the filename
        a = os.path.join(myPath, module)  
        
        targetsList = []
        #print("Started Reading JSON file which contains multiple JSON document")
        with open(a) as f:
            for jsonObj in f:
                targetsListDict = json.loads(jsonObj)
                targetsList.append(targetsListDict)
        
        i = 0
        nrow = len(targetsList)
        file_data = []
        while i < nrow:
            new_row = [targetsList[i]['id'], 
                   targetsList[i]['name'], 
                   targetsList[i]['therapeuticAreas']]
            file_data.append(new_row)
            
            i = i+1
            #print(targetsList[i-1])
        
        if first:
            all_data = file_data
            first = False
            
        
        else:
            all_data = all_data+file_data
            
df_diseases = pd.DataFrame(all_data, columns = col_names) 
df_diseases


,id,name,therapeuticAreas
0,EFO_0000182,hepatocellular carcinoma,"[MONDO_0045024, EFO_0001379, EFO_0010282]"
1,EFO_0002616,macroglobulinemia,"[OTAR_0000018, OTAR_0000020]"
2,EFO_0004703,age at menarche,[EFO_0001444]
3,EFO_0004806,asbestos exposure measurement,[EFO_0001444]
4,EFO_0004867,vitamin E measurement,[EFO_0001444]
...,...,...,...
24708,Orphanet_276058,Genetic neurodegenerative disease with dementia,"[OTAR_0000018, EFO_0000618]"
24709,Orphanet_52901,Isolated follicle stimulating hormone deficiency,"[OTAR_0000017, OTAR_0000018, EFO_0001379]"
24710,Orphanet_73220,X-linked intellectual disability - hypotonic face,[OTAR_0000018]
24711,Orphanet_79095,Congenital bile acid synthesis defect type 4,"[OTAR_0000018, OTAR_0000020, EFO_0001379, EFO_..."


In [4]:
df_diseases.loc[df_diseases.id == 'MONDO_0045024','name'].tolist()[0] #look up one disease ID

'cancer or benign tumor'

In [5]:
one_row = df_diseases[df_diseases['id']=='EFO_0000182']

print(one_row['therapeuticAreas'].tolist()[0])

def create_TA_EFO_list(one_row):
    
    name_list = one_row['therapeuticAreas'].tolist()[0]
    empty_list = []
    
    for i in name_list:
        #print(i)
        name_i = df_diseases.loc[df_diseases.id == i,'name'].tolist()[0]
        #print(name_i)
        empty_list = empty_list+[name_i]
        
    return empty_list

A = create_TA_EFO_list(one_row)
A

['MONDO_0045024', 'EFO_0001379', 'EFO_0010282']


['cancer or benign tumor',
 'endocrine system disease',
 'gastrointestinal disease']

In [6]:
def update_EFO_lists_df(df):
    new_column = []
    
    all_terms = df['id'].copy()
    
    for i in all_terms:
        row = df[df['id']==i]
        text_list = create_TA_EFO_list(row)
        new_column.append(text_list)
    return new_column

new_col = update_EFO_lists_df(df_diseases)
df_diseases['therapeuticAreas_text'] = new_col
df_diseases

,id,name,therapeuticAreas,therapeuticAreas_text
0,EFO_0000182,hepatocellular carcinoma,"[MONDO_0045024, EFO_0001379, EFO_0010282]","[cancer or benign tumor, endocrine system dise..."
1,EFO_0002616,macroglobulinemia,"[OTAR_0000018, OTAR_0000020]","[genetic, familial or congenital disease, nutr..."
2,EFO_0004703,age at menarche,[EFO_0001444],[measurement]
3,EFO_0004806,asbestos exposure measurement,[EFO_0001444],[measurement]
4,EFO_0004867,vitamin E measurement,[EFO_0001444],[measurement]
...,...,...,...,...
24708,Orphanet_276058,Genetic neurodegenerative disease with dementia,"[OTAR_0000018, EFO_0000618]","[genetic, familial or congenital disease, nerv..."
24709,Orphanet_52901,Isolated follicle stimulating hormone deficiency,"[OTAR_0000017, OTAR_0000018, EFO_0001379]","[reproductive system or breast disease, geneti..."
24710,Orphanet_73220,X-linked intellectual disability - hypotonic face,[OTAR_0000018],"[genetic, familial or congenital disease]"
24711,Orphanet_79095,Congenital bile acid synthesis defect type 4,"[OTAR_0000018, OTAR_0000020, EFO_0001379, EFO_...","[genetic, familial or congenital disease, nutr..."


In [7]:
df_diseases.to_csv("DiseasesInformation_OpenTargets_06092023.csv", sep=',')

In [8]:
#load matched cell x gene disease terms
df_diseases_ed = pd.read_csv("DiseasesInformation_OpenTargets_06142023_ed.csv")

In [9]:
df_diseases_ed.head()

,Unnamed: 0,id,name,therapeuticAreas,therapeuticAreas_text,cxg_matched_id
0,0,EFO_0000182,hepatocellular carcinoma,"['MONDO_0045024', 'EFO_0001379', 'EFO_0010282']","['cancer or benign tumor', 'endocrine system d...",MONDO_0007256
1,1,EFO_0002616,macroglobulinemia,"['OTAR_0000018', 'OTAR_0000020']","['genetic, familial or congenital disease', 'n...",MONDO_0005190
2,2,EFO_0004703,age at menarche,['EFO_0001444'],['measurement'],EFO_0004703
3,3,EFO_0004806,asbestos exposure measurement,['EFO_0001444'],['measurement'],EFO_0004806
4,4,EFO_0004867,vitamin E measurement,['EFO_0001444'],['measurement'],EFO_0004867


In [10]:
new_col[471]

['musculoskeletal or connective tissue disease',
 'urinary system disease',
 'immune system disease',
 'hematologic disease']

In [11]:
df_diseases[df_diseases['id']=='MONDO_0007915']

,id,name,therapeuticAreas,therapeuticAreas_text
471,MONDO_0007915,systemic lupus erythematosus,"[OTAR_0000006, EFO_0009690, EFO_0000540, EFO_0...","[musculoskeletal or connective tissue disease,..."


In [12]:
#df_drugs.groupby(['phase'], sort=False).size()

In [13]:
#data_dic = {
#    "a": ['a','a','b','c'],
#    "b": [3,3,4,5],
#    "c": [6,7,8,9]
#}
#df1 = pd.DataFrame(data_dic)

#data_dic = {
#    "a": ['a','b','b','c','d'],
#    "d": [3,3,4,4,5],
#    "e": [10,10,10, 12,13]
#}
#df2 = pd.DataFrame(data_dic)
#df1

In [14]:
#df2

In [15]:
#df = pd.merge(df1, df2, on='a' , how='inner')
#df

In [16]:
# now aggregate association evidences

In [17]:
#associations by data type direct
myPath = 'platform/23.02/output/etl/json/associationByDatatypeDirect'

#col_names = ['drugId', 'targetId','approvedSymbol', 'diseaseId', 'label', 'phase', 'drugType', 'mechanismOfAction']
col_names = ['targetId', 'diseaseId', 'datatypeId', 'score', 'evidenceCount']


allFiles = os.listdir(myPath)
first = True
for module in allFiles:
    # get one file in directory
    if 'part-' in module: #if the word module is in the filename
        a = os.path.join(myPath, module)  
        
        targetsList = []
        #print("Started Reading JSON file which contains multiple JSON document")
        with open(a) as f:
            for jsonObj in f:
                targetsListDict = json.loads(jsonObj)
                targetsList.append(targetsListDict)
        
        i = 0
        nrow = len(targetsList)
        file_data = []
        while i < nrow:
            new_row = [targetsList[i]['targetId'], 
                   targetsList[i]['diseaseId'], 
                   targetsList[i]['datatypeId'], 
                   targetsList[i]['score'], 
                   targetsList[i]['evidenceCount']]
            file_data.append(new_row)
            i = i+1
        
        if first:
            all_data = file_data
            first = False
        
        else:
            all_data = all_data+file_data
            
df_associations = pd.DataFrame(all_data, columns = col_names) 
df_associations

,targetId,diseaseId,datatypeId,score,evidenceCount
0,ENSG00000151151,MONDO_0018698,genetic_association,0.607931,1
1,ENSG00000010282,MONDO_0018701,literature,0.024317,1
2,ENSG00000105357,MONDO_0018701,literature,0.012159,1
3,ENSG00000157119,MONDO_0018701,literature,0.012159,1
4,ENSG00000007062,MONDO_0018715,literature,0.042555,1
...,...,...,...,...,...
2145822,ENSG00000188582,EFO_0004308,genetic_association,0.100365,1
2145823,ENSG00000197296,EFO_0004308,genetic_association,0.034327,1
2145824,ENSG00000275066,EFO_0004308,genetic_association,0.047564,1
2145825,ENSG00000278540,EFO_0004308,genetic_association,0.043192,1


In [18]:
df_associations['targetId-diseaseId'] = df_associations.targetId.str.cat(df_associations.diseaseId, sep='-')

In [19]:
u = df_associations.pivot(index='targetId-diseaseId', columns='datatypeId', values='score')
df_1=u.reset_index()
df_1

datatypeId,targetId-diseaseId,affected_pathway,animal_model,genetic_association,known_drug,literature,rna_expression,somatic_mutation
0,ENSG00000000003-EFO_0000191,NaN,NaN,NaN,NaN,NaN,0.013836,NaN
1,ENSG00000000003-EFO_0000199,NaN,NaN,NaN,NaN,NaN,0.094351,NaN
2,ENSG00000000003-EFO_0000228,NaN,NaN,NaN,NaN,0.463547,NaN,NaN
3,ENSG00000000003-EFO_0000232,NaN,NaN,NaN,NaN,0.349560,NaN,NaN
4,ENSG00000000003-EFO_0000272,NaN,NaN,NaN,NaN,NaN,0.164222,NaN
...,...,...,...,...,...,...,...,...
2046199,ENSG00000291237-Orphanet_99015,NaN,0.29515,NaN,NaN,NaN,NaN,NaN
2046200,ENSG00000291281-EFO_0004616,NaN,NaN,NaN,NaN,0.018238,NaN,NaN
2046201,ENSG00000291281-MONDO_0005178,NaN,NaN,NaN,NaN,0.127497,NaN,NaN
2046202,ENSG00000291300-EFO_0002517,NaN,NaN,NaN,NaN,0.607931,NaN,NaN


In [20]:
df_1.fillna(0, inplace=True)
df_2 = df_1
df_4 = df_1[df_1['known_drug']!=0.00]
df_3=df_4.reset_index()

In [21]:
df_2

datatypeId,targetId-diseaseId,affected_pathway,animal_model,genetic_association,known_drug,literature,rna_expression,somatic_mutation
0,ENSG00000000003-EFO_0000191,0.0,0.00000,0.0,0.0,0.000000,0.013836,0.0
1,ENSG00000000003-EFO_0000199,0.0,0.00000,0.0,0.0,0.000000,0.094351,0.0
2,ENSG00000000003-EFO_0000228,0.0,0.00000,0.0,0.0,0.463547,0.000000,0.0
3,ENSG00000000003-EFO_0000232,0.0,0.00000,0.0,0.0,0.349560,0.000000,0.0
4,ENSG00000000003-EFO_0000272,0.0,0.00000,0.0,0.0,0.000000,0.164222,0.0
...,...,...,...,...,...,...,...,...
2046199,ENSG00000291237-Orphanet_99015,0.0,0.29515,0.0,0.0,0.000000,0.000000,0.0
2046200,ENSG00000291281-EFO_0004616,0.0,0.00000,0.0,0.0,0.018238,0.000000,0.0
2046201,ENSG00000291281-MONDO_0005178,0.0,0.00000,0.0,0.0,0.127497,0.000000,0.0
2046202,ENSG00000291300-EFO_0002517,0.0,0.00000,0.0,0.0,0.607931,0.000000,0.0


In [22]:
#df_3.to_csv("TargetDiseasePairs_hasKnownDrug_OpenTargets_06092023.csv", sep=',')

df_2['targetId'], df_2['diseaseId'] = df_2['targetId-diseaseId'].str.split('-',1).str
df_diseases_ed = pd.read_csv("DiseasesInformation_OpenTargets_06142023_ed.csv")

cxg_rename_df = df_diseases_ed[["id", "cxg_matched_id"]].copy()
cxg_rename_df = cxg_rename_df.rename(columns={'id': 'diseaseId', 'cxg_matched_id': 'cxg_matched_id'})

mergedData2 = df_2.merge(right=cxg_rename_df, on ='diseaseId', how ="left")
mergedData2

#mergedData2.to_csv("TargetDiseasePairs_OpenTargets_cellXgeneID_12072023.csv", sep=',')

/opt/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  This is separate from the ipykernel package so we can avoid doing imports until


,targetId-diseaseId,affected_pathway,animal_model,genetic_association,known_drug,literature,rna_expression,somatic_mutation,targetId,diseaseId,cxg_matched_id
0,ENSG00000000003-EFO_0000191,0.0,0.00000,0.0,0.0,0.000000,0.013836,0.0,ENSG00000000003,EFO_0000191,MONDO_0007650
1,ENSG00000000003-EFO_0000199,0.0,0.00000,0.0,0.0,0.000000,0.094351,0.0,ENSG00000000003,EFO_0000199,MONDO_0004958
2,ENSG00000000003-EFO_0000228,0.0,0.00000,0.0,0.0,0.463547,0.000000,0.0,ENSG00000000003,EFO_0000228,MONDO_0004970
3,ENSG00000000003-EFO_0000232,0.0,0.00000,0.0,0.0,0.349560,0.000000,0.0,ENSG00000000003,EFO_0000232,MONDO_0004972
4,ENSG00000000003-EFO_0000272,0.0,0.00000,0.0,0.0,0.000000,0.164222,0.0,ENSG00000000003,EFO_0000272,MONDO_0019781
...,...,...,...,...,...,...,...,...,...,...,...
2046199,ENSG00000291237-Orphanet_99015,0.0,0.29515,0.0,0.0,0.000000,0.000000,0.0,ENSG00000291237,Orphanet_99015,Orphanet_99015
2046200,ENSG00000291281-EFO_0004616,0.0,0.00000,0.0,0.0,0.018238,0.000000,0.0,ENSG00000291281,EFO_0004616,MONDO_0005416
2046201,ENSG00000291281-MONDO_0005178,0.0,0.00000,0.0,0.0,0.127497,0.000000,0.0,ENSG00000291281,MONDO_0005178,MONDO_0005178
2046202,ENSG00000291300-EFO_0002517,0.0,0.00000,0.0,0.0,0.607931,0.000000,0.0,ENSG00000291300,EFO_0002517,MONDO_0005184
